In [2]:
import numpy as np
import matplotlib.pyplot as plt

from utils_3D import c, w, cs2, Q, D, K0, K1, compute_feq, rmsre

In [3]:
def compute_rho_u(num_samples, rho_min=0.95, rho_max=1.05, u_abs_min=0.0, u_abs_max=0.01):
    
    rho   = np.random.uniform(rho_min, rho_max, size=num_samples)    
    u_abs = np.random.uniform(u_abs_min, u_abs_max, size=num_samples)
    
    # Random direction in 3D (uniform on sphere)
    phi   = np.random.uniform(0, 2*np.pi, size=num_samples)
    theta = np.arccos(np.random.uniform(-1, 1, size=num_samples))
    
    ux = u_abs * np.sin(theta) * np.cos(phi)
    uy = u_abs * np.sin(theta) * np.sin(phi)
    uz = u_abs * np.cos(theta)
    u  = np.array([ux, uy, uz]).transpose()
    
    return rho, u


def compute_f_rand(num_samples, sigma_min, sigma_max):

    f_rand = np.zeros((num_samples, Q))

    if sigma_min==sigma_max:
        sigma = sigma_min*np.ones(num_samples)
    else:
        sigma = np.random.uniform(sigma_min, sigma_max, size=num_samples)    

    for i in range(num_samples):
        f_rand[i,:] = np.random.normal(0, sigma[i], size=(1,Q))

        rho_hat = np.sum(f_rand[i,:])
        u_hat = np.einsum("j,jk->k", f_rand[i], c)

        f_rand[i,:] = f_rand[i,:] - K0*rho_hat - K1*np.einsum("k,jk->j", u_hat, c)
         
    return f_rand


def compute_f_pre_f_post(f_eq, f_neq, tau_min=1, tau_max=1):
    
    tau    = np.random.uniform(tau_min, tau_max, size=f_eq.shape[0])
    f_pre  = f_eq + f_neq
    
    f_post = f_pre + 1/tau[:,None]*(f_eq - f_pre)

    return tau, f_pre, f_post


def delete_negative_samples(n_samples, f_eq, f_pre, f_post):
    
    i_neg_f_eq   = np.where(np.sum(f_eq  <0,axis=1) > 0)[0]
    i_neg_f_pre  = np.where(np.sum(f_pre <0,axis=1) > 0)[0]
    i_neg_f_post = np.where(np.sum(f_post<0,axis=1) > 0)[0]

    i_neg_f = np.concatenate( (i_neg_f_pre, i_neg_f_post, i_neg_f_eq) )
    
    f_eq   = np.delete(np.copy(f_eq)  , i_neg_f, 0)
    f_pre  = np.delete(np.copy(f_pre) , i_neg_f, 0)
    f_post = np.delete(np.copy(f_post), i_neg_f, 0)
    
    return f_eq, f_pre, f_post

In [4]:
#####################################
# settings 

n_samples = 100_000

u_abs_min = 1e-15
u_abs_max = 0.01
sigma_min = 1e-15 
sigma_max = 5e-4  

#####################################

fPreLst  = np.empty( (n_samples, Q) )
fPostLst = np.empty( (n_samples, Q) )
fEqLst   = np.empty( (n_samples, Q) )

#####################################

idx = 0

# loop until we get n_samples without negative populations
while idx < n_samples: 
    
    # get random values for macroscopic quantities
    rho, u = compute_rho_u(n_samples)

    rho = rho[:,np.newaxis,np.newaxis]
    ux  = u[:,0][:,np.newaxis,np.newaxis]
    uy  = u[:,1][:,np.newaxis,np.newaxis]
    uz  = u[:,2][:,np.newaxis,np.newaxis]

    # compute the equilibrium distribution
    f_eq  = np.zeros((n_samples, 1, 1, Q))
    f_eq  = compute_feq(f_eq, rho, ux, uy, uz)[:,0,0,:]
    
    # compute a random non equilibrium part
    f_neq = compute_f_rand(n_samples, sigma_min, sigma_max)   
    
    # apply BGK to f_pre = f_eq + f_neq
    tau , f_pre, f_post = compute_f_pre_f_post(f_eq, f_neq)
    
    # remove negative elements
    f_eq, f_pre, f_post = delete_negative_samples(n_samples, f_eq, f_pre, f_post)
    
    # accumulate 
    non_negatives = f_pre.shape[0]
    
    idx1        = min(idx+non_negatives, n_samples)
    to_be_added = min(n_samples-idx, non_negatives)
    
    fPreLst[ idx:idx1] = f_pre[ :to_be_added]
    fPostLst[idx:idx1] = f_post[:to_be_added]
    fEqLst[  idx:idx1] = f_eq[  :to_be_added]
    
    idx = idx + non_negatives

In [5]:
# store data on file

np.savez('example_dataset_3D.npz', 
        f_pre  = fPreLst,
        f_post = fPostLst,
        f_eq   = fEqLst
       )

In [6]:
# print(u_hat)